**Goals**

Crowdsourcing annotations has become a fundamental aspect of NLP research. The goal of this hands-on exercise is to explore the ethical implications of soliciting crowdsourced data, specifically social biases that may emerge when asking for generated sentences.

**Overview**

In this exercise, you will perform a “bias audit” of an NLP dataset produced by crowdsourcing. You will attempt to measure the presence of social stereotypes in this dataset that may have harmful effects if used to train classifiers in downstream tasks.

We will use pointwise mutual information (PMI) to find which associations are being made with identity labels. PMI can be used as a measure of word association in a corpus, i.e. how frequently two words co-occur above what might just be expected based on their frequencies.

See the [PMI Wikipedia page](https://en.wikipedia.org/wiki/Pointwise_mutual_information) for more details. Here we use PMI to measure which words co-occur with labels for identities. This allows us to see associations that may perpetuate stereotypes.

After this analysis, you will present specific examples from the data that you speculate could be particularly biased and problematic.

*Assignment design credits:* [11-830 Computational Ethics in NLP Course](https://maartensap.com/11830/Spring2024/hw1.html)

**Load and prepare dataset**

In [1]:
#List of identity labels (based on Rudinger et al. 2017)
#More comprehensive list can be obtained from Jain et al. 2024
with open("ss-data/identity_labels.txt", 'r') as f:
    identity_labels = f.read().split("\n")
print(identity_labels)

['woman', 'women', 'man', 'men', 'girl', 'girls', 'boy', 'boys', 'she', 'he', 'her', 'him', 'his', 'female', 'male', 'mother', 'father', 'sister', 'brother', 'daughter', 'son', 'feminine', 'masculine', 'androgynous', 'trans', 'transgender', 'transsexual', 'nonbinary', 'non-binary', 'two-spirit', 'hijra', 'genderqueer', 'black', 'asian', 'hispanic', 'white', 'african', 'american', 'latino', 'latina', 'caucasian', 'africans', 'middle-eastern', 'australian', 'australians', 'asians', 'european', 'europeans', 'chinese', 'indian', 'indonesian', 'brazilian', 'pakistani', 'bangladeshi', 'russian', 'nigerian', 'japanese', 'mexican', 'filipino ', 'vietnamese ', 'german', 'egyptian', 'ethiopian', 'turkish', 'iranian', 'thai', 'congolese', 'french', 'british ', 'italian', 'korean', 'burmese', 'canadian ', 'australian ', 'spanish', 'dutch', 'swiss', 'saudi', 'argentinian ', 'taiwanese ', 'swedish ', 'belgian', 'polish', 'israeli', 'irish', 'greek', 'ukrainian ', 'jamaican ', 'mongolian', 'armenian'

In [2]:
#Load SNLI dataset
import pandas as pd
snli_data = pd.read_json(path_or_buf="ss-data/snli_1.0_train.jsonl", lines=True)
pd.set_option('display.max_colwidth', 0)

snli_data

,annotator_labels,captionID,gold_label,pairID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],3416050480.jpg#4,neutral,3416050480.jpg#4r1n,A person on a horse jumps over a broken down airplane.,( ( ( A person ) ( on ( a horse ) ) ) ( ( jumps ( over ( a ( broken ( down airplane ) ) ) ) ) . ) ),(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN on) (NP (DT a) (NN horse)))) (VP (VBZ jumps) (PP (IN over) (NP (DT a) (JJ broken) (JJ down) (NN airplane)))) (. .))),A person is training his horse for a competition.,( ( A person ) ( ( is ( ( training ( his horse ) ) ( for ( a competition ) ) ) ) . ) ),(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) (VP (VBG training) (NP (PRP$ his) (NN horse)) (PP (IN for) (NP (DT a) (NN competition))))) (. .)))
1,[contradiction],3416050480.jpg#4,contradiction,3416050480.jpg#4r1c,A person on a horse jumps over a broken down airplane.,( ( ( A person ) ( on ( a horse ) ) ) ( ( jumps ( over ( a ( broken ( down airplane ) ) ) ) ) . ) ),(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN on) (NP (DT a) (NN horse)))) (VP (VBZ jumps) (PP (IN over) (NP (DT a) (JJ broken) (JJ down) (NN airplane)))) (. .))),"A person is at a diner, ordering an omelette.","( ( A person ) ( ( ( ( is ( at ( a diner ) ) ) , ) ( ordering ( an omelette ) ) ) . ) )","(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) (PP (IN at) (NP (DT a) (NN diner))) (, ,) (S (VP (VBG ordering) (NP (DT an) (NN omelette))))) (. .)))"
2,[entailment],3416050480.jpg#4,entailment,3416050480.jpg#4r1e,A person on a horse jumps over a broken down airplane.,( ( ( A person ) ( on ( a horse ) ) ) ( ( jumps ( over ( a ( broken ( down airplane ) ) ) ) ) . ) ),(ROOT (S (NP (NP (DT A) (NN person)) (PP (IN on) (NP (DT a) (NN horse)))) (VP (VBZ jumps) (PP (IN over) (NP (DT a) (JJ broken) (JJ down) (NN airplane)))) (. .))),"A person is outdoors, on a horse.","( ( A person ) ( ( ( ( is outdoors ) , ) ( on ( a horse ) ) ) . ) )","(ROOT (S (NP (DT A) (NN person)) (VP (VBZ is) (ADVP (RB outdoors)) (, ,) (PP (IN on) (NP (DT a) (NN horse)))) (. .)))"
3,[neutral],2267923837.jpg#2,neutral,2267923837.jpg#2r1n,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at camera ) ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smiling) (CC and) (VBG waving) (PP (IN at) (NP (NN camera))))))),They are smiling at their parents,( They ( are ( smiling ( at ( their parents ) ) ) ) ),(ROOT (S (NP (PRP They)) (VP (VBP are) (VP (VBG smiling) (PP (IN at) (NP (PRP$ their) (NNS parents)))))))
4,[entailment],2267923837.jpg#2,entailment,2267923837.jpg#2r1e,Children smiling and waving at camera,( Children ( ( ( smiling and ) waving ) ( at camera ) ) ),(ROOT (NP (S (NP (NNP Children)) (VP (VBG smiling) (CC and) (VBG waving) (PP (IN at) (NP (NN camera))))))),There are children present,( There ( ( are children ) present ) ),(ROOT (S (NP (EX There)) (VP (VBP are) (NP (NNS children)) (ADVP (RB present)))))
...,...,...,...,...,...,...,...,...,...,...
550147,[contradiction],2267923837.jpg#3,contradiction,2267923837.jpg#3r1c,Four dirty and barefooted children.,( ( ( ( Four dirty ) and ) ( barefooted children ) ) . ),(ROOT (NP (NP (CD Four) (NNS dirty)) (CC and) (NP (VBN barefooted) (NNS children)) (. .))),four kids won awards for 'cleanest feet',( ( four kids ) ( ( won awards ) ( ( ( for ` ) ( cleanest feet ) ) ' ) ) ),(ROOT (S (NP (CD four) (NNS kids)) (VP (VBD won) (NP (NNS awards)) (PP (IN for) (`` `) (NP (JJ cleanest) (NNS feet)) ('' ')))))
550148,[neutral],2267923837.jpg#3,neutral,2267923837.jpg#3r1n,Four dirty and barefooted children.,( ( ( ( Four dirty ) and ) ( barefooted children ) ) . ),(ROOT (NP (NP (CD Four) (NNS dirty)) (CC and) (NP (VBN barefooted) (NNS children)) (. .))),"four homeless children had their shoes stolen, so their feet are dirty.","( ( ( ( ( ( four ( homeless children ) ) ( had ( ( their shoes ) stolen ) ) ) , ) so ) ( ( their feet ) ( are dirty ) ) ) . )","(ROOT (S (S (NP (CD four) (JJ homeless) (NNS children)) (

In [3]:
#De-duplicate
snli_data_sub = snli_data[['sentence1','sentence2']]
print(len(snli_data_sub))
snli_data_sub.drop_duplicates(subset=['sentence1', 'sentence2'], keep='last',inplace=True)
print(len(snli_data_sub))

550152
549526


/tmp/ipykernel_3081672/1005867814.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub.drop_duplicates(subset=['sentence1', 'sentence2'], keep='last',inplace=True)


**Data preparation**

In [4]:
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter
nltk.download('punkt')

from nltk.corpus import stopwords
import string
nltk.download('stopwords')

from itertools import combinations
from tqdm import tqdm

[nltk_data] Downloading package punkt to
[nltk_data]     /home/t-jainprachi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/t-jainprachi/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
#util functions
def get_removal_list():
  stop_words = set(stopwords.words('english'))
  string.punctuation = string.punctuation +'"'+'-'+'''+'''+'—'
  string.punctuation
  removal_list = list(stop_words) + list(string.punctuation)+ ['lt','rt']
  return removal_list

removal_list = get_removal_list()

def remove_stopwords(sent_tokens):
    for punct in list(string.punctuation):
        sent_tokens = [word.strip(punct).strip() for word in sent_tokens]

    #removal_list = get_removal_list()
    filtered_words = [word.strip() for word in sent_tokens if word not in removal_list]

    filtered_words = [ele for ele in filtered_words if ele]
    return filtered_words

# def remove_stopwords(sent_tokens):
#     filtered_words = [word for word in sent_tokens if word not in removal_list]
#     return filtered_words

# def remove_stopwords(sent_tokens):
#     # for punct in list(string.punctuation):
#     #     sent_tokens = [word.strip(punct).strip() for word in sent_tokens]

#     #removal_list = get_removal_list()
#     filtered_words = [word.strip().strip("'").strip('"').strip(".").strip("!").strip() for word in sent_tokens if word not in removal_list]

#     filtered_words = [ele for ele in filtered_words if ele]
#     return filtered_words

In [6]:
#Lowercase
snli_data_sub['sentence1'] = snli_data_sub['sentence1'].map(str.lower)
snli_data_sub['sentence2'] = snli_data_sub['sentence2'].map(str.lower)
snli_data_sub

/tmp/ipykernel_3081672/91705368.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence1'] = snli_data_sub['sentence1'].map(str.lower)
/tmp/ipykernel_3081672/91705368.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence2'] = snli_data_sub['sentence2'].map(str.lower)


,sentence1,sentence2
0,a person on a horse jumps over a broken down airplane.,a person is training his horse for a competition.
1,a person on a horse jumps over a broken down airplane.,"a person is at a diner, ordering an omelette."
2,a person on a horse jumps over a broken down airplane.,"a person is outdoors, on a horse."
3,children smiling and waving at camera,they are smiling at their parents
4,children smiling and waving at camera,there are children present
...,...,...
550147,four dirty and barefooted children.,four kids won awards for 'cleanest feet'
550148,four dirty and barefooted children.,"four homeless children had their shoes stolen, so their feet are dirty."
550149,a man is surfing in a bodysuit in beautiful blue water.,a man in a bodysuit is competing in a surfing competition.
550150,a man is surfing in a bodysuit in beautiful blue water.,a man in a business suit is heading to a board meeting.


In [7]:
#Tokenize
snli_data_sub['sentence1_token'] = snli_data_sub['sentence1'].apply(nltk.word_tokenize)
snli_data_sub['sentence2_token'] = snli_data_sub['sentence2'].apply(nltk.word_tokenize)

snli_data_sub

/tmp/ipykernel_3081672/2199046777.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence1_token'] = snli_data_sub['sentence1'].apply(nltk.word_tokenize)
/tmp/ipykernel_3081672/2199046777.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence2_token'] = snli_data_sub['sentence2'].apply(nltk.word_tokenize)


,sentence1,sentence2,sentence1_token,sentence2_token
0,a person on a horse jumps over a broken down airplane.,a person is training his horse for a competition.,"[a, person, on, a, horse, jumps, over, a, broken, down, airplane, .]","[a, person, is, training, his, horse, for, a, competition, .]"
1,a person on a horse jumps over a broken down airplane.,"a person is at a diner, ordering an omelette.","[a, person, on, a, horse, jumps, over, a, broken, down, airplane, .]","[a, person, is, at, a, diner, ,, ordering, an, omelette, .]"
2,a person on a horse jumps over a broken down airplane.,"a person is outdoors, on a horse.","[a, person, on, a, horse, jumps, over, a, broken, down, airplane, .]","[a, person, is, outdoors, ,, on, a, horse, .]"
3,children smiling and waving at camera,they are smiling at their parents,"[children, smiling, and, waving, at, camera]","[they, are, smiling, at, their, parents]"
4,children smiling and waving at camera,there are children present,"[children, smiling, and, waving, at, camera]","[there, are, children, present]"
...,...,...,...,...
550147,four dirty and barefooted children.,four kids won awards for 'cleanest feet',"[four, dirty, and, barefooted, children, .]","[four, kids, won, awards, for, 'cleanest, feet, ']"
550148,four dirty and barefooted children.,"four homeless children had their shoes stolen, so their feet are dirty.","[four, dirty, and, barefooted, children, .]","[four, homeless, children, had, their, shoes, stolen, ,, so, their, feet, are, dirty, .]"
550149,a man is surfing in a bodysuit in beautiful blue water.,a man in a bodysuit is competing in a surfing competition.,"[a, man, is, surfing, in, a, bodysuit, in, beautiful, blue, water, .]","[a, man, in, a, bodysuit, is, competing, in, a, surfing, competition, .]"
550150,a man is surfing in a bodysuit in beautiful blue water.,a man in a business suit is heading to a board meeting.,"[a, man, is, surfing, in, a, bodysuit, in, beautiful, blue, water, .]","[a, man, in, a, business, suit, is, heading, to, a, board, meeting, .]"


In [8]:
#Remove stop-words
from time import time
from tqdm import tqdm
tqdm.pandas()

snli_data_sub['sentence1_token_nostopwords'] = snli_data_sub['sentence1_token'].progress_apply(remove_stopwords)
snli_data_sub['sentence2_token_nostopwords'] = snli_data_sub['sentence2_token'].progress_apply(remove_stopwords)

snli_data_sub

100%|██████████| 549526/549526 [00:22<00:00, 23915.07it/s]
/tmp/ipykernel_3081672/1713195379.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence1_token_nostopwords'] = snli_data_sub['sentence1_token'].progress_apply(remove_stopwords)
100%|██████████| 549526/549526 [00:14<00:00, 37139.04it/s]
/tmp/ipykernel_3081672/1713195379.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence2_token_nostopwords'] = snli_data_sub['sentence2_token'].progress_apply(remove_stopwor

,sentence1,sentence2,sentence1_token,sentence2_token,sentence1_token_nostopwords,sentence2_token_nostopwords
0,a person on a horse jumps over a broken down airplane.,a person is training his horse for a competition.,"[a, person, on, a, horse, jumps, over, a, broken, down, airplane, .]","[a, person, is, training, his, horse, for, a, competition, .]","[person, horse, jumps, broken, airplane]","[person, training, horse, competition]"
1,a person on a horse jumps over a broken down airplane.,"a person is at a diner, ordering an omelette.","[a, person, on, a, horse, jumps, over, a, broken, down, airplane, .]","[a, person, is, at, a, diner, ,, ordering, an, omelette, .]","[person, horse, jumps, broken, airplane]","[person, diner, ordering, omelette]"
2,a person on a horse jumps over a broken down airplane.,"a person is outdoors, on a horse.","[a, person, on, a, horse, jumps, over, a, broken, down, airplane, .]","[a, person, is, outdoors, ,, on, a, horse, .]","[person, horse, jumps, broken, airplane]","[person, outdoors, horse]"
3,children smiling and waving at camera,they are smiling at their parents,"[children, smiling, and, waving, at, camera]","[they, are, smiling, at, their, parents]","[children, smiling, waving, camera]","[smiling, parents]"
4,children smiling and waving at camera,there are children present,"[children, smiling, and, waving, at, camera]","[there, are, children, present]","[children, smiling, waving, camera]","[children, present]"
...,...,...,...,...,...,...
550147,four dirty and barefooted children.,four kids won awards for 'cleanest feet',"[four, dirty, and, barefooted, children, .]","[four, kids, won, awards, for, 'cleanest, feet, ']","[four, dirty, barefooted, children]","[four, kids, awards, cleanest, feet]"
550148,four dirty and barefooted children.,"four homeless children had their shoes stolen, so their feet are dirty.","[four, dirty, and, barefooted, children, .]","[four, homeless, children, had, their, shoes, stolen, ,, so, their, feet, are, dirty, .]","[four, dirty, barefooted, children]","[four, homeless, children, shoes, stolen, feet, dirty]"
550149,a man is surfing in a bodysuit in beautiful blue water.,a man in a bodysuit is competing in a surfing competition.,"[a, man, is, surfing, in, a, bodysuit, in, beautiful, blue, water, .]","[a, man, in, a, bodysuit, is, competing, in, a, surfing, competition, .]","[man, surfing, bodysuit, beautiful, blue, water]","[man, bodysuit, competing, surfing, competition]"
550150,a man is surfing in a bodysuit in beautiful blue water.,a man in a business suit is heading to a board meeting.,"[a, man, is, surfing, in, a, bodysuit, in, beautiful, blue, water, .]","[a, man, in, a, business, suit, is, heading, to, a, board, meeting, .]","[man, surfing, bodysuit, beautiful, blue, water]","[man, business, suit, heading, board, meeting]"


In [9]:
print(len(snli_data_sub))

snli_data_sub['sentence1_token_nostopwords_str'] = snli_data_sub['sentence1_token_nostopwords'].apply(lambda x: ' '.join(x))
snli_data_sub['sentence2_token_nostopwords_str'] = snli_data_sub['sentence2_token_nostopwords'].apply(lambda x: ' '.join(x))

snli_data_sub.drop_duplicates(subset=['sentence1_token_nostopwords_str', 'sentence2_token_nostopwords_str'], keep='last', inplace=True)
print(len(snli_data_sub))

549526


/tmp/ipykernel_3081672/817760109.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence1_token_nostopwords_str'] = snli_data_sub['sentence1_token_nostopwords'].apply(lambda x: ' '.join(x))
/tmp/ipykernel_3081672/817760109.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub['sentence2_token_nostopwords_str'] = snli_data_sub['sentence2_token_nostopwords'].apply(lambda x: ' '.join(x))


547558


/tmp/ipykernel_3081672/817760109.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  snli_data_sub.drop_duplicates(subset=['sentence1_token_nostopwords_str', 'sentence2_token_nostopwords_str'], keep='last', inplace=True)


**Word association analysis**

$PMI(w_i, w_j)$ = $log_2 \frac{p(w_i, w_j)}{P(w_i)P(w_j)}$ = $log_2\frac{N\cdot c(w_i, w_j)}{c(w_i)c(w_j)}$


In [10]:
#Computing Unigram frequency
corpus = []
for sent in (snli_data_sub['sentence1_token_nostopwords'].tolist() + snli_data_sub['sentence2_token_nostopwords'].tolist()):
  corpus+=(sent)
unigram_frequency = Counter(corpus)
unigram_frequency.most_common(20)

[('man', 265138),
 ('woman', 137105),
 ('two', 121714),
 ('people', 120803),
 ('wearing', 80592),
 ('young', 61353),
 ('men', 60853),
 ('playing', 59219),
 ('girl', 59094),
 ('boy', 58042),
 ('white', 56793),
 ('shirt', 56204),
 ('black', 54824),
 ('dog', 53688),
 ('sitting', 53544),
 ('blue', 49040),
 ('standing', 46189),
 ('red', 43130),
 ('group', 43061),
 ('walking', 38681)]

In [11]:
#Remove words with less than 15 freq
from itertools import dropwhile
print(len(unigram_frequency))
for key, count in dropwhile(lambda key_count: key_count[1] >= 15, unigram_frequency.most_common()):
    del unigram_frequency[key]
print(len(unigram_frequency))

36193
10638


In [12]:
choice = "separate_prem_hypo"

if choice == "combine_prem_hypo":
  snli_data_sub['sentence1_sentence2_token_nostopwords'] = snli_data_sub['sentence1_token_nostopwords'] + snli_data_sub['sentence2_token_nostopwords']
  ##MAIN
  bigram_doc=[]
  for ele in tqdm(snli_data_sub['sentence1_sentence2_token_nostopwords'].tolist()):
    bigram_doc.append(list(set(list(nltk.bigrams(ele)))))
  bigram=[]
  for doc in bigram_doc:
    bigram += doc
  bigram_frequency = Counter(bigram)
  bigram_frequency
elif choice == "separate_prem_hypo":
  ##
  snli_data_sub_s1 = snli_data_sub[['sentence1_token_nostopwords']].copy()
  snli_data_sub_s2 = snli_data_sub[['sentence2_token_nostopwords']].copy()

  print("S1",len(snli_data_sub_s1))
  snli_data_sub_s1['sentence1_token_nostopwords_str'] = snli_data_sub_s1['sentence1_token_nostopwords'].apply(lambda x: ' '.join(x))
  snli_data_sub_s1.drop_duplicates(subset=['sentence1_token_nostopwords_str'], keep='last',inplace=True)
  print("S1",len(snli_data_sub_s1))

  print("S2",len(snli_data_sub_s2))
  snli_data_sub_s2['sentence2_token_nostopwords_str'] = snli_data_sub_s2['sentence2_token_nostopwords'].apply(lambda x: ' '.join(x))
  snli_data_sub_s2.drop_duplicates(subset=['sentence2_token_nostopwords_str'], keep='last',inplace=True)
  print("S2",len(snli_data_sub_s2))
  ##MAIN
  bigram_doc=[]
  for ele in tqdm(snli_data_sub_s1['sentence1_token_nostopwords'].tolist()+snli_data_sub_s2['sentence2_token_nostopwords'].tolist()):
    bigram_doc.append(list(set(list(nltk.bigrams(ele)))))
  bigram=[]
  for doc in bigram_doc:
    bigram += doc
  bigram_frequency = Counter(bigram)
  bigram_frequency

S1 547558
S1 149532
S2 547558
S2 423258


100%|██████████| 572790/572790 [00:01<00:00, 349408.41it/s]


In [13]:
#Remove words-pair with less than 10 freq
from itertools import dropwhile
print(len(bigram_frequency))
for key, count in dropwhile(lambda key_count: key_count[1] >= 10, bigram_frequency.most_common()):
    del bigram_frequency[key]
print(len(bigram_frequency))

659784
29093


$PMI(w_i, w_j)$ = $log_2 \frac{p(w_i, w_j)}{P(w_i)P(w_j)}$ = $log_2\frac{N\cdot c(w_i, w_j)}{c(w_i)c(w_j)}$


In [14]:
#Calculate PMI
import math

def pmi(word1, word2, unigram_freq, bigram_freq):
  if word1 in unigram_freq.keys() and word2 in unigram_freq.keys():
    if (word1, word2) in bigram_freq.keys() or (word2, word1) in bigram_freq.keys():
      prob_word1 = unigram_freq[word1] / float(sum(unigram_freq.values()))
      prob_word2 = unigram_freq[word2] / float(sum(unigram_freq.values()))
      prob_word1_word2 = (bigram_freq[(word1, word2)]) / float(sum(bigram_freq.values()))
      if prob_word1_word2 >0:
        return math.log(prob_word1_word2/float(prob_word1*prob_word2),2)


In [15]:
identity_labels

['woman',
 'women',
 'man',
 'men',
 'girl',
 'girls',
 'boy',
 'boys',
 'she',
 'he',
 'her',
 'him',
 'his',
 'female',
 'male',
 'mother',
 'father',
 'sister',
 'brother',
 'daughter',
 'son',
 'feminine',
 'masculine',
 'androgynous',
 'trans',
 'transgender',
 'transsexual',
 'nonbinary',
 'non-binary',
 'two-spirit',
 'hijra',
 'genderqueer',
 'black',
 'asian',
 'hispanic',
 'white',
 'african',
 'american',
 'latino',
 'latina',
 'caucasian',
 'africans',
 'middle-eastern',
 'australian',
 'australians',
 'asians',
 'european',
 'europeans',
 'chinese',
 'indian',
 'indonesian',
 'brazilian',
 'pakistani',
 'bangladeshi',
 'russian',
 'nigerian',
 'japanese',
 'mexican',
 'filipino ',
 'vietnamese ',
 'german',
 'egyptian',
 'ethiopian',
 'turkish',
 'iranian',
 'thai',
 'congolese',
 'french',
 'british ',
 'italian',
 'korean',
 'burmese',
 'canadian ',
 'australian ',
 'spanish',
 'dutch',
 'swiss',
 'saudi',
 'argentinian ',
 'taiwanese ',
 'swedish ',
 'belgian',
 'polish

In [16]:
def Sort(tuple):
    # reverse = None (Sorts in Ascending order)
    return(sorted(tuple, key = lambda a: a[1], reverse = True))

def get_pmi(identity_label,unigram_frequency,bigram_frequency):
  pmi_identity_label=[]
  for word in tqdm(unigram_frequency.keys()):
    pmi_score = pmi(word1=identity_label.lower(), word2=word,unigram_freq=unigram_frequency,bigram_freq=bigram_frequency)
    if pmi_score:
      pmi_identity_label.append((word,pmi_score))

  return Sort(pmi_identity_label)


In [30]:
from collections import defaultdict as dd
pmi_identity_label = dd(list)
for identity_label in (identity_labels):
  print(identity_label)
  for word in tqdm(unigram_frequency.keys()):
    pmi_score = pmi(word1=identity_label, word2=word,unigram_freq=unigram_frequency,bigram_freq=bigram_frequency)
    #print("PMI:",pmi_score)
    if pmi_score:
      pmi_identity_label[identity_label].append((word,pmi_score))

woman


100%|██████████| 10638/10638 [00:01<00:00, 9923.30it/s]


women


100%|██████████| 10638/10638 [00:00<00:00, 26005.84it/s]


man


...

teenager


100%|██████████| 10638/10638 [00:00<00:00, 743290.84it/s]


teenagers


100%|██████████| 10638/10638 [00:00<00:00, 355620.60it/s]


100%|██████████| 10638/10638 [00:00<00:00, 2339503.25it/s]


In [31]:
for identity in pmi_identity_label.keys():
    pmi_identity_label[identity] = Sort(pmi_identity_label[identity])

for key in pmi_identity_label.keys():
    print(key, pmi_identity_label[key][:20])

woman [('buys', 5.864070952771102), ('ordered', 5.826092903207355), ('considering', 5.7607391652132565), ('knits', 5.489748035551119), ('weaves', 5.480287706302052), ('modeling', 5.416784763995894), ('applies', 5.40228519430078), ('flirting', 5.213251369910762), ('sips', 5.200651333131129), ('dries', 5.1881603889479315), ('sews', 5.169624437510505), ('frowns', 5.088624715397017), ('admires', 5.012718382538053), ('feeds', 4.944603357613442), ('shares', 4.817322693579623), ('likes', 4.769342664677802), ('calls', 4.748460835434819), ('owns', 4.714752959939076), ('ironing', 4.6907902876506915), ('gave', 4.61729808527163)]
women [('gossiping', 8.184286926573977), ('bikinis', 6.583842998409607), ('discuss', 5.990145687710843), ('knitting', 5.624395126157468), ('strolling', 5.616330851158511), ('sunbathing', 5.605914235213173), ('discussing', 5.573367554751044), ('chatting', 5.504525353057125), ('chat', 5.490390054299658), ('arguing', 5.3394242707431605), ('jog', 5.245883502903738), ('dressin

In [17]:
man_pmi_data = get_pmi('man',unigram_frequency,bigram_frequency)
woman_pmi_data = get_pmi('woman',unigram_frequency,bigram_frequency)
gay_pmi_data = get_pmi('gay',unigram_frequency,bigram_frequency)


men_pmi_data = get_pmi('men',unigram_frequency,bigram_frequency)
women_pmi_data = get_pmi('women',unigram_frequency,bigram_frequency)
lesbian_pmi_data = get_pmi('lesbian',unigram_frequency,bigram_frequency)

lady_pmi_data = get_pmi('lady',unigram_frequency,bigram_frequency)

100%|██████████| 10638/10638 [00:00<00:00, 55964.32it/s]


In [18]:
def filter_pmi_set(pmi_data,threshold):
    return set([ele[0] for ele in pmi_data if ele[1]>threshold])
    

male_pmi_set = filter_pmi_set(man_pmi_data, 2.0).union(filter_pmi_set(men_pmi_data, 2.0))
female_pmi_set = filter_pmi_set(woman_pmi_data, 2.0).union(filter_pmi_set(women_pmi_data, 2.0)).union(filter_pmi_set(lady_pmi_data, 2.0))
lgbt_pmi_set = filter_pmi_set(gay_pmi_data, 2.0).union(filter_pmi_set(lesbian_pmi_data,2.0))

In [19]:
male_pmi_set

{'accepts',
 'accidentally',
 'acting',
 'actually',
 'adjusting',
 'adjusts',
 'admires',
 'admiring',
 'advertising',
 'afraid',
 'afro',
 'aiming',
 'almost',
 'angrily',
 'appear',
 'approaches',
 'approaching',
 'argue',
 'arguing',
 'army',
 'arranging',
 'arrested',
 'asked',
 'asking',
 'asks',
 'asleep',
 'ate',
 'attempt',
 'attempting',
 'attempts',
 'attending',
 'attends',
 'awake',
 'backpack',
 'backpacks',
 'bad',
 'baggy',
 'baking',
 'balancing',
 'balcony',
 'bar',
 'barbecuing',
 'beard',
 'beating',
 'begging',
 'beige',
 'bending',
 'bends',
 'bent',
 'beret',
 'best',
 'bicycles',
 'bicycling',
 'biking',
 'black',
 'blowing',
 'blows',
 'blue',
 'boat',
 'bought',
 'bowler',
 'bowls',
 'boxing',
 'breakdances',
 'breakdancing',
 'breaking',
 'breaks',
 'bringing',
 'brings',
 'broke',
 'brothers',
 'browsing',
 'brushing',
 'build',
 'builds',
 'bundled',
 'bungee',
 'buried',
 'burning',
 'burns',
 'business',
 'buying',
 'buys',
 'ca',
 'calling',
 'calls',
 '

In [20]:
female_pmi_set

{'adjusting',
 'adjusts',
 'admires',
 'admiring',
 'advertising',
 'afraid',
 'alone',
 'also',
 'angry',
 'applies',
 'applying',
 'arguing',
 'asking',
 'attending',
 'attends',
 'baking',
 'balances',
 'balancing',
 'ballet',
 'bathing',
 'beige',
 'bending',
 'bends',
 'bent',
 'biking',
 'bikini',
 'bikinis',
 'black',
 'blond-hair',
 'blonde',
 'blow',
 'blowing',
 'blows',
 'blue',
 'bought',
 'braids',
 'breaking',
 'breaks',
 'bright',
 'bringing',
 'broke',
 'brown',
 'browsing',
 'brushes',
 'brushing',
 'bundled',
 'buying',
 'buys',
 'ca',
 'calling',
 'calls',
 'camping',
 'cane',
 'carried',
 'carries',
 'carry',
 'carrying',
 'celebrates',
 'celebrating',
 'chat',
 'chatting',
 'checking',
 'checks',
 'cheering',
 'chopping',
 'cleaning',
 'cleans',
 'climbs',
 'close',
 'coffee',
 'collecting',
 'colorful',
 'compete',
 'competes',
 'competing',
 'considering',
 'conversation',
 'converse',
 'conversing',
 'cook',
 'cooking',
 'cooks',
 'costumes',
 'covering',
 'cove

In [21]:
male_pmi_set - female_pmi_set

{'accepts',
 'accidentally',
 'acting',
 'actually',
 'afro',
 'aiming',
 'almost',
 'angrily',
 'appear',
 'approaches',
 'approaching',
 'argue',
 'army',
 'arranging',
 'arrested',
 'asked',
 'asks',
 'asleep',
 'ate',
 'attempt',
 'attempting',
 'attempts',
 'awake',
 'backpack',
 'backpacks',
 'bad',
 'baggy',
 'balcony',
 'bar',
 'barbecuing',
 'beard',
 'beating',
 'begging',
 'beret',
 'best',
 'bicycles',
 'bicycling',
 'boat',
 'bowler',
 'bowls',
 'boxing',
 'breakdances',
 'breakdancing',
 'brings',
 'brothers',
 'build',
 'builds',
 'bungee',
 'buried',
 'burning',
 'burns',
 'business',
 'came',
 'camouflage',
 'canoe',
 'carefully',
 'carves',
 'carving',
 'cast',
 'casual',
 'catches',
 'catching',
 'caught',
 'celebrate',
 'changes',
 'changing',
 'chased',
 'chasing',
 'checkered',
 'cherry',
 'chiseling',
 'chops',
 'clean',
 'clearing',
 'climb',
 'climbed',
 'climbing',
 'closes',
 'closing',
 'collared',
 'collects',
 'completely',
 'concentrating',
 'conducting',

In [22]:
female_pmi_set - male_pmi_set

{'alone',
 'also',
 'angry',
 'applies',
 'applying',
 'balances',
 'ballet',
 'bathing',
 'bikini',
 'bikinis',
 'blond-hair',
 'blonde',
 'blow',
 'braids',
 'bright',
 'brown',
 'brushes',
 'cheering',
 'close',
 'coffee',
 'colorful',
 'considering',
 'costumes',
 'covering',
 'cross',
 'crying',
 'curly',
 'date',
 'daughters',
 'displaying',
 'displays',
 'dress',
 'dresses',
 'dries',
 'drops',
 'drying',
 'embrace',
 'embracing',
 'ethnic',
 'exiting',
 'fancy',
 'floral',
 'flowered',
 'formal',
 'get',
 'gossiping',
 'green',
 'hair',
 'happy',
 'headscarf',
 'heels',
 'hospital',
 'hug',
 'hula',
 'husband',
 'indoors',
 'jog',
 'kimono',
 'kiss',
 'kitchen',
 'knees',
 'knit',
 'knits',
 'laugh',
 'laundry',
 'leopard',
 'library',
 'made',
 'maroon',
 'modeling',
 'multicolored',
 'ordered',
 'pajamas',
 'pass',
 'pet',
 'photographed',
 'pink',
 'purple',
 'purse',
 'red',
 'red-hair',
 'roller',
 'rolling',
 'saying',
 'screaming',
 'shares',
 'shop',
 'shopping',
 'shor

In [23]:
lgbt_pmi_set - male_pmi_set.union(female_pmi_set)

{'couple', 'men', 'pride', 'rights'}

In [27]:
search_term = "dirty"
mask = snli_data["sentence2"].str.contains(search_term, case=False, na=False)
pd.set_option('display.max_colwidth', 0)
#pd.set_option("display.max_rows", None)
snli_data[mask][["gold_label","sentence1","sentence2"]]



,gold_label,sentence1,sentence2
1857,neutral,a woman on a yellow shirt is on the floor.,A woman lying on her shirt so her pants wont get dirty.
5352,neutral,An old man wit a straw hat and umbrella is smiling while sitting and watching something in the distance.,The hat is dirty.
5358,contradiction,A young boy wearing a red shirt with yellow writing on it and a red sports cap is squatted in shallow water and touching a yellow pail filled with sand.,"The little kid with a red shirt and yellow letters, stood on a foot stool to reach both his hands into the dirty water in the sink."
5518,contradiction,A father or family friend playing with baby in a very clean looking house.,The house is dirty looking.
6208,neutral,A man in orange shorts sits on some steps in front of a big blue door.,A young man in shorts sits on some dirty steps.
...,...,...,...
544008,contradiction,Children in festive attire walking with others on a street.,Kids in dirty blue jeans walk on the street.
547285,neutral,Three people with white gloves on picking up trash on a beach.,Three people pick up dirty needles on a beach.
548798,entailment,Two people are cleaning the road.,Two people are cleaning the dirty road.
550146,entailment,Four dirty and barefooted children.,four children have dirty feet.


In [28]:
search_term = "extravagant"
mask = snli_data["sentence1"].str.contains(search_term, case=False, na=False)
pd.set_option('display.max_colwidth', 0)
snli_data[mask][["gold_label","sentence1","sentence2"]]

,gold_label,sentence1,sentence2
20205,neutral,A Indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,The woman is preparing for her wedding.
20206,contradiction,A Indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,"A woman dresses in western fashion, with unadorned fingers and head."
20207,neutral,A Indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,An Indian lady is wearing a beautiful outfit.
20208,entailment,A Indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,An Indian woman is wearing traditional styles.
20209,neutral,A Indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,A woman shows off to a crowd of people.
20210,neutral,A Indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,A woman is ready for a national cultural celebration.
20211,contradiction,A Indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,A Japanese woman is wearing a kimono.
20212,entailment,A Indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,"A very colorfully robed female with fingers painted red, who's culture appears to be Indian, has on an elaborate head piece."
20213,entailment,A Indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,"A woman is dressed in traditional, ethnic clothing."
20214,contradiction,A Indian women displaying her cultural heritage with painted red fingers and an extravagant head piece.,A woman wearing an old-fashioned apron holds an apple pie in her hands.


In [29]:
search_term = "terrorist"
mask = snli_data["sentence2"].str.contains(search_term, case=False, na=False)
pd.set_option('display.max_colwidth', 0)
#pd.set_option("display.max_rows", None)
snli_data[mask][["gold_label","sentence1","sentence2"]]



,gold_label,sentence1,sentence2
51534,neutral,People praying by a car while another reads something from a large piece of paper.,The people read the newspaper in horror of the recent American terrorist attacks.
54040,contradiction,A young woman is standing in a darkly-lit room with two red lamps hanging overhead.,George Washington has released a video claiming responsibility for the most recent terrorist attack.
82780,contradiction,Children sitting and writing.,Children writing bomb threats to mail to terrorists.
97940,contradiction,"A man holds up a tiny sign reading ""I'm a photographer not a terrorist.""",A cat is a terrorist.
98016,contradiction,"A man is holding a souvenir that says ""I'm a photographer not a terrorist"" at what appears to be a rally somewhere in Europe.","A man is stealing someones souvenir that says ""I'm a terrorist not a photographer please shoot me""."
98018,entailment,"A man is holding a souvenir that says ""I'm a photographer not a terrorist"" at what appears to be a rally somewhere in Europe.","Ar a rally in Europe a man is holding a souvenir that says ""I'm a photographer not a terrorist""."
117324,neutral,A man holds a flying national flag amid debris and smoke.,The flag is raised as a symbol of strength after the terrorist attack.
119273,neutral,A group of people are congregating outside with a wave flying and a banner of foreign writing on it.,The people are terrorists
130270,neutral,Several Muslim worshipers march towards Mecca.,The Muslims are terrorists.
135622,neutral,Military personnel patrol an area.,Soldiers are looking for an escaped terrorist.
